In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [30]:
pd.options.display.max_colwidth = 250
data = pd.read_csv('cleaned_AllPeople.csv', low_memory=False)
sessions = pd.read_csv('Cleaned_Events_With_SessionID.csv', low_memory=False)
#metrics_labels = pd.read_csv('labeled_with_metrics.csv')

In [31]:
sessions.drop('Unnamed: 0', axis=1, inplace=True)
data.drop('Unnamed: 0', axis=1, inplace=True)

In [32]:
#metrics = metrics_labels.drop('churned', axis=1)
#metrics.drop('Unnamed: 0', axis=1, inplace=True)

In [33]:
#metrics

In [34]:
num_sessions = sessions[['distinct_id', 'session_id']].groupby('distinct_id').agg('count').rename({'session_id' : 'num_sessions'}, axis=1)
num_sessions

,num_sessions
distinct_id,
00000000-0000-0000-0000-000000000000,2
000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,97
001210DC-54C8-43AD-B295-148F47818391,61
0019C3B9-FA62-4AB4-A895-1390A1FA818C,1
0025A8A4-2590-4ECA-8CE8-419D710AE46F,64
...,...
wandaandreu@gmail.com_5053,5
willwojt@gmail.com_6868,178
wolphramite@gmail.com_291,4


In [35]:
sessions

,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,FirstTimeUploaded,Signup With,imageAdded,commentAdded,commentLength,ImageUploaded,ae_total_app_session_length,name_y,ae_total_app_sessions,timezone,last_seen,ios_app_version,ios_version,ios_device_model,ios_lib_version,ae_first_app_open_date,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown,session_id,time_delta
0,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-02-13 15:52:50,-1.0,Mumbai,Maharashtra,IN,1,1,NaN,Foodie,NaN,2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTL,0,0,0,0,0,-1,0.0
1,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-03-08 20:43:54,-1.0,Ahmedabad,Gujarat,IN,1,1,NaN,Foodie,NaN,2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTL,0,0,0,0,0,-1,0.0
2,AppOpen,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:32,-1.0,Palo Alto,California,US,1,0,NaN,Google,NaN,2,0.0,NaN,0.905,Lily Aronovitz,11.0,America/Los_Angeles,2019-08-24 21:57:06+00:00,1.9.6,12.4,"iPhone10,6",3.4.4,2019-08-23 18:48:32+00:00,1 days 03:08:34.000000000,82.272727,US,1,0,0,0,1,1,0.0
3,Visit Signup/Login Page,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:35,-1.0,Palo Alto,California,US,1,0,NaN,Google,NaN,2,0.0,NaN,0.905,Lily Aronovitz,11.0,America/Los_Angeles,2019-08-24 21:57:06+00:00,1.9.6,12.4,"iPhone10,6",3.4.4,2019-08-23 18:48:32+00:00,1 days 03:08:34.000000000,82.272727,US,1,0,0,0,1,1,0.0
4,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:49:02,30.1,Palo Alto,California,US,1,0,NaN,Google,NaN,2,0.0,NaN,0.905,Lily Aronovitz,11.0,America/Los_Angeles,2019-08-24 21:57:06+00:00,1.9.6,12.4,"iPhone10,6",3.4.4,2019-08-23 18:48:32+00:00,1 days 03:08:34.000000000,82.272727,US,1,0,0,0,1,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190366,Feed - Scroll Down,xbarbarazhong@gmail.com_6704,2019-12-12 14:29:23,-1.0,Portland,Oregon,US,1,0,NaN,Foodie,NaN,2,0.0,NaN,NaN,Barbara Zhong,NaN,America/Los_Angeles,2019-12-13 04:27:46+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,1,1,0,0,0,29813,3.0
190367,Profile - Change Section,xbarbarazhong@gmail.com_6704,2019-12-12 14:29:28,-1.0,Portland,Oregon,US,1,0,NaN,Foodie,NaN,2,0.0,NaN,NaN,Barbara Zhong,NaN,America/Los_Angeles,2019-12-13 04:27:46+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,1,1,0,0,0,29813,3.0
190368,$ae_session,xbarbarazhong@gmail.com_6704,2019-12-12 14:30:54,190.4,Portland,Oregon,US,1,1,NaN,Foodie,NaN,2,0.0,NaN,NaN,Barbara Zhong,NaN,America/Los_Angeles,2019-12-13 04:27:46+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,1,1,0,0,0,29813,3.0
190369,AppOpen,yoon.s.jeong@gmail.com_6485,2019-10-09 11:12:29,-1.0,Chicago,Illinois,US,1,1,NaN,Foodie,NaN,2,0.0,NaN,NaN,Yoon Jeong,NaN,America/Chicago,2019-10-09 23:12:29+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,1,0,0,1,0,29814,0.0


In [36]:
data = data.merge(num_sessions, how='left', left_on='distinct_id', right_index=True)

In [37]:
#data = data.merge(metrics, how='left', left_on='distinct_id', right_on='distinct_id')
data['commentAdded'].value_counts()

2    185185
0      3708
1      1478
Name: commentAdded, dtype: int64

In [38]:
data['commentLength'].value_counts()

0.0      188893
8.0         102
6.0          53
4.0          40
5.0          39
          ...  
327.0         1
258.0         1
245.0         1
240.0         1
254.0         1
Name: commentLength, Length: 333, dtype: int64

# Jake Changes (Aggregating outside columns)

In [39]:
aggregated_columns = data.copy()[['distinct_id','commentLength', 'commentAdded']]
commentAdded = aggregated_columns['commentAdded']

commentFix = []
for i in commentAdded:
    if i == 2:
        commentFix.append(0)
    elif i == 1:
        commentFix.append(1)
    else:
        commentFix.append(0)
commentFix = np.array(commentFix)
aggregated_columns['commentAdded'] = commentFix

In [40]:
sum_columns = aggregated_columns.copy()[['distinct_id', 'commentAdded']]
mean_columns = aggregated_columns.copy()[['distinct_id','commentLength']]

In [41]:
comment_Added_DF = sum_columns.groupby('distinct_id').agg(sum)
comment_Length_DF = mean_columns.groupby('distinct_id').agg(np.mean)

In [42]:
def replace_first(row):
    try:
        if (pd.isnull(row["ae_first_app_open_date"])):
            return row["time"]
        else:
            return row["ae_first_app_open_date"]
    except:
        return 0
    
def replace_last_seen(row):
    try:
        if (row["last_time"] > row["last_seen"]):
            return row["last_time"]
        elif (pd.isnull(row["last_seen"])): 
            return row["last_time"]
        else:
            return row["last_seen"]
    except:
        return 0
    
def last_time_add(row):
    di = row["distinct_id"]
    return keep_last[keep_last["distinct_id"] == di ].iloc[0]["time"]

In [43]:
pd.set_option('display.max_columns', None)

sorted_data = data.copy()
sorted_data["last_seen"] = pd.to_datetime(sorted_data['last_seen'], utc = True)
sorted_data["time"] = pd.to_datetime(sorted_data['time'], utc = True)
sorted_data["ae_first_app_open_date"] = pd.to_datetime(sorted_data["ae_first_app_open_date"], utc = True)
sorted_data = sorted_data.sort_values("time", ascending = True)
keep_last = sorted_data.copy()

sorted_data = sorted_data.drop_duplicates(subset = "distinct_id", keep = "first")
keep_last = keep_last.drop_duplicates(subset = "distinct_id", keep = "last")

sorted_data["last_time"] = sorted_data.apply(last_time_add, axis = 1)
sorted_data["last_seen"] = sorted_data.apply(replace_last_seen, axis = 1)
sorted_data["ae_first_app_open_date"] = sorted_data.apply(replace_first, axis = 1)

date_data = sorted_data.copy()
average_total_app_sessions = np.mean(date_data["ae_total_app_sessions"])
date_data['ae_total_app_sessions'] = date_data['ae_total_app_sessions'].fillna(average_total_app_sessions)
date_data['active_timespan'] = date_data["last_seen"] - date_data["ae_first_app_open_date"]
date_data['active_timespan'] = [elem.total_seconds() for elem in date_data['active_timespan']]
date_data['active_timespan'] = date_data['active_timespan'] / (60 * 60 * 24)

date_data['sessions_per_day'] = date_data['num_sessions'] / 180
date_data["sessions_per_day"].replace({np.inf: 0}, inplace=True)
date_data = date_data.sort_values("distinct_id", ascending = True)

In [44]:
filled_data = date_data.copy()
# filled_data = filled_data[["distinct_id", "active_timespan", "ae_total_app_sessions", "sessions_per_day"]]

In [45]:
list(filled_data.columns)

['name_x',
 'distinct_id',
 'time',
 'ae_session_length',
 'city',
 'region',
 'mp_country_code',
 'locationSetting',
 'notificationSettings',
 'FirstTimeUploaded',
 'Signup With',
 'imageAdded',
 'commentAdded',
 'commentLength',
 'ImageUploaded',
 'ae_total_app_session_length',
 'name_y',
 'ae_total_app_sessions',
 'timezone',
 'last_seen',
 'ios_app_version',
 'ios_version',
 'ios_device_model',
 'ios_lib_version',
 'ae_first_app_open_date',
 'active_timespan',
 'average_session_time',
 'country',
 'US',
 'Unknown',
 'Facebook',
 'Foodie',
 'Google',
 'num_sessions',
 'last_time',
 'sessions_per_day']

# Jake (selecting the columns we want)

In [46]:
selected_filled = filled_data.copy()[['distinct_id', 'locationSetting', 'notificationSettings', 'timezone', 'active_timespan', 'average_session_time', 'US', 'Facebook', 'Foodie', 'Google', 'Unknown', 'num_sessions', 'sessions_per_day']]
selected_filled['timezone'].value_counts()

America/New_York               1970
America/Los_Angeles             917
America/Chicago                 867
Asia/Kolkata                    783
America/Phoenix                  41
                               ... 
Asia/Kuwait                       1
Europe/Tirane                     1
America/St_Thomas                 1
America/Kentucky/Louisville       1
Asia/Makassar                     1
Name: timezone, Length: 79, dtype: int64

## One-Hot Encoding Timezone

In [47]:
old_Timezone = selected_filled.copy()['timezone']
NY = []
LA = []
Ch = []
Kol = []
De = []
other_timezone = []

for index in old_Timezone:
    if index == 'America/New_York':
        NY.append(1)
        LA.append(0)
        Ch.append(0)
        Kol.append(0)
        De.append(0)
        other_timezone.append(0)
    elif index == 'America/Los_Angeles':
        NY.append(0)
        LA.append(1)
        Ch.append(0)
        Kol.append(0)
        De.append(0)
        other_timezone.append(0)
    elif index == 'America/Chicago':
        NY.append(0)
        LA.append(0)
        Ch.append(1)
        Kol.append(0)
        De.append(0)
        other_timezone.append(0)
    elif index == 'Asia/Kolkata':
        NY.append(0)
        LA.append(0)
        Ch.append(0)
        Kol.append(1)
        De.append(0)
        other_timezone.append(0)
    elif index == 'America/Denver':
        NY.append(0)
        LA.append(0)
        Ch.append(0)
        Kol.append(0)
        De.append(1)
        other_timezone.append(0)
    else:
        NY.append(0)
        LA.append(0)
        Ch.append(0)
        Kol.append(0)
        De.append(0)
        other_timezone.append(1)
        
NY = np.array(NY)
LA = np.array(LA)
Ch = np.array(Ch)
Kol = np.array(Kol)
De = np.array(De)
other_timezone = np.array(other_timezone)

In [48]:
selected_filled = selected_filled.drop('timezone', axis=1)
selected_filled['America/New_York'] = NY
selected_filled['America/Los_Angeles'] = LA
selected_filled['America/Chicago'] = Ch
selected_filled['Asia/Kolkata'] = Kol
selected_filled['America/Denver'] = De
selected_filled['other_timezone'] = other_timezone

In [49]:
label_Ready = selected_filled.merge(comment_Added_DF, how='left', left_on='distinct_id', right_index=True)
label_Ready = label_Ready.merge(comment_Length_DF, how='left', left_on='distinct_id', right_index=True)
label_Ready

,distinct_id,locationSetting,notificationSettings,active_timespan,average_session_time,US,Facebook,Foodie,Google,Unknown,num_sessions,sessions_per_day,America/New_York,America/Los_Angeles,America/Chicago,Asia/Kolkata,America/Denver,other_timezone,commentAdded,commentLength
83729,00000000-0000-0000-0000-000000000000,1,0,24.202130,NaN,0,0,1,0,0,2,0.011111,0,0,0,0,0,1,0,0.000000
84492,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,1,1,1.130949,82.272727,1,0,0,1,1,97,0.538889,0,1,0,0,0,0,0,0.000000
55831,001210DC-54C8-43AD-B295-148F47818391,1,0,7.158310,47.600000,1,0,0,1,1,61,0.338889,0,0,1,0,0,0,0,0.000000
102081,0019C3B9-FA62-4AB4-A895-1390A1FA818C,1,1,0.000000,NaN,1,0,1,0,0,1,0.005556,0,0,0,0,0,1,0,0.000000
33171,0025A8A4-2590-4ECA-8CE8-419D710AE46F,1,0,113.687361,176.750000,1,0,1,0,1,64,0.355556,1,0,0,0,0,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70158,wandaandreu@gmail.com_5053,1,0,188.791597,NaN,1,0,1,0,1,5,0.027778,1,0,0,0,0,0,0,0.000000
87817,willwojt@gmail.com_6868,1,0,37.992164,NaN,1,0,1,0,1,178,0.988889,1,0,0,0,0,0,5,2.252809
92278,wolphramite@gmail.com_291,1,0,79.125498,NaN,1,0,1,0,1,4,0.022222,1,0,0,0,0,0,0,0.000000
101276,xbarbarazhong@gmail.com_6704,1,0,0.584016,NaN,1,0,1,0,1,21,0.116667,0,1,0,0,0,0,0,0.000000


# Churn Label

In [50]:
def labeling(row):
    if (row["active_timespan"] > 0 and row["active_timespan"] < 150 and row["sessions_per_day"] < 0.14):
        return 1
    else:
        return 0

In [51]:
labeled_data = label_Ready.copy()
labeled_data["churned"] = labeled_data.apply(labeling, axis = 1)

## labeled_data is the final dataset, cleaned and with churn label

In [52]:
labeled_data

,distinct_id,locationSetting,notificationSettings,active_timespan,average_session_time,US,Facebook,Foodie,Google,Unknown,num_sessions,sessions_per_day,America/New_York,America/Los_Angeles,America/Chicago,Asia/Kolkata,America/Denver,other_timezone,commentAdded,commentLength,churned
83729,00000000-0000-0000-0000-000000000000,1,0,24.202130,NaN,0,0,1,0,0,2,0.011111,0,0,0,0,0,1,0,0.000000,1
84492,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,1,1,1.130949,82.272727,1,0,0,1,1,97,0.538889,0,1,0,0,0,0,0,0.000000,0
55831,001210DC-54C8-43AD-B295-148F47818391,1,0,7.158310,47.600000,1,0,0,1,1,61,0.338889,0,0,1,0,0,0,0,0.000000,0
102081,0019C3B9-FA62-4AB4-A895-1390A1FA818C,1,1,0.000000,NaN,1,0,1,0,0,1,0.005556,0,0,0,0,0,1,0,0.000000,0
33171,0025A8A4-2590-4ECA-8CE8-419D710AE46F,1,0,113.687361,176.750000,1,0,1,0,1,64,0.355556,1,0,0,0,0,0,0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70158,wandaandreu@gmail.com_5053,1,0,188.791597,NaN,1,0,1,0,1,5,0.027778,1,0,0,0,0,0,0,0.000000,0
87817,willwojt@gmail.com_6868,1,0,37.992164,NaN,1,0,1,0,1,178,0.988889,1,0,0,0,0,0,5,2.252809,0
92278,wolphramite@gmail.com_291,1,0,79.125498,NaN,1,0,1,0,1,4,0.022222,1,0,0,0,0,0,0,0.000000,1
101276,xbarbarazhong@gmail.com_6704,1,0,0.584016,NaN,1,0,1,0,1,21,0.116667,0,1,0,0,0,0,0,0.000000,1


# EDA

In [53]:
labeled_data
#plt.hist(labeled_data['churned'])

,distinct_id,locationSetting,notificationSettings,active_timespan,average_session_time,US,Facebook,Foodie,Google,Unknown,num_sessions,sessions_per_day,America/New_York,America/Los_Angeles,America/Chicago,Asia/Kolkata,America/Denver,other_timezone,commentAdded,commentLength,churned
83729,00000000-0000-0000-0000-000000000000,1,0,24.202130,NaN,0,0,1,0,0,2,0.011111,0,0,0,0,0,1,0,0.000000,1
84492,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,1,1,1.130949,82.272727,1,0,0,1,1,97,0.538889,0,1,0,0,0,0,0,0.000000,0
55831,001210DC-54C8-43AD-B295-148F47818391,1,0,7.158310,47.600000,1,0,0,1,1,61,0.338889,0,0,1,0,0,0,0,0.000000,0
102081,0019C3B9-FA62-4AB4-A895-1390A1FA818C,1,1,0.000000,NaN,1,0,1,0,0,1,0.005556,0,0,0,0,0,1,0,0.000000,0
33171,0025A8A4-2590-4ECA-8CE8-419D710AE46F,1,0,113.687361,176.750000,1,0,1,0,1,64,0.355556,1,0,0,0,0,0,0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70158,wandaandreu@gmail.com_5053,1,0,188.791597,NaN,1,0,1,0,1,5,0.027778,1,0,0,0,0,0,0,0.000000,0
87817,willwojt@gmail.com_6868,1,0,37.992164,NaN,1,0,1,0,1,178,0.988889,1,0,0,0,0,0,5,2.252809,0
92278,wolphramite@gmail.com_291,1,0,79.125498,NaN,1,0,1,0,1,4,0.022222,1,0,0,0,0,0,0,0.000000,1
101276,xbarbarazhong@gmail.com_6704,1,0,0.584016,NaN,1,0,1,0,1,21,0.116667,0,1,0,0,0,0,0,0.000000,1


In [54]:
# labeled_data.drop(0, axis=0, inplace=True)
# labeled_data.drop(102081, axis=0, inplace=True)
# labeled_data.drop('ae_total_app_sessions', axis=1, inplace=True)
#labeled_data

In [56]:
#labeled_data = labeled_data.merge(metrics, how='left', left_on='distinct_id', right_on='distinct_id')

In [57]:
labeled_data.to_csv('labeled_data.csv')

In [ ]:
plt.xlabel('Active Timespan (Days)')
plt.ylabel('Sessions per Day')
plt.ylim((0, 10))
plt.xlim((0, 400))
plt.title('Timespan vs. Sessions Per Day')
plt.scatter(labeled_data['active_timespan'], labeled_data['sessions_per_day'])

In [ ]:
plt.xlabel('Active Timespan (Days)')
plt.ylabel('Number of sessions')
plt.title('Timespan vs. Num sessions')
plt.scatter(labeled_data['active_timespan'], labeled_data['ae_total_app_sessions'])

# Pranav's Other Plots

In [ ]:
plt.title('Histogram of Total App Sessions')
plt.hist(id_sessions['ae_total_app_sessions'])

In [ ]:
plt.title('Histogram with Bins 30-500 Size 10, Total App Sessions')
plt.hist(id_sessions['ae_total_app_sessions'], bins=np.linspace(30, 500, 48))

# Comparing Both Results Before Replacing Last_Seen (DON'T RUN)

In [ ]:
erica = date_data.copy()
pranav = sorted_id.copy()
erica.sort_values("distinct_id", ascending = True)
pranav.sort_values("distinct_id", ascending = True)

In [20]:
erica["sessions_per_day"].equals(pranav["sessions_per_day"])

False

In [25]:
import math
math.isnan
one = erica["sessions_per_day"].tolist()
two = pranav["sessions_per_day"].tolist()
for i in range(len(one)):
    o = one[i]
    t = two[i]
    if (one[i] != two[i]):
        if (math.isnan(o) == False):
            if (abs(o - t) > .1):
                print(o, t)

0.07278842354177645 24.531639758206442
28.432669743902608 28.62024638457417
0.4900072581207263 28.620246384574187
0.07970300149199487 24.531639758206435
21.055767094918338 22.46111695334107
11.327481848956525 28.620246384574177
34.49461021715357 35.20221076685976
0.10954724150571979 24.531639758206435
1.6586572703588247 28.62024638457418
0.08807520294213118 4.26179604261796
47.04174228675136 47.39007221866716
0.4223504978400627 84.00194448945578
0.054142424184744097 0.28945184219849396
0.027654150283022944 5.142857142857142
10.45040568548097 11.020674106173274
0.47962805035023925 2.014818812471189


In [26]:
erica[erica["sessions_per_day"] == 0.07278842354177645]

,Unnamed: 0,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,FirstTimeUploaded,Signup With,imageAdded,commentAdded,commentLength,ImageUploaded,ae_total_app_session_length,name_y,ae_total_app_sessions,timezone,last_seen,ios_app_version,ios_version,ios_device_model,ios_lib_version,ae_first_app_open_date,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown,active_timespan_days,sessions_per_day
70325,39167,$ae_updated,1a28969580acaa3f,2019-08-06 14:18:28.000000,NaN,Crystal Lake,Illinois,US,1,0,NaN,Foodie,NaN,2,0.0,NaN,NaN,Natalie K,14.310123,America/Detroit,2020-02-19 04:40:52+00:00,NaN,NaN,NaN,NaN,2019-08-06 14:18:28+00:00,196 days 14:22:24,NaN,US,1,0,1,0,0,196.598889,0.072788


In [27]:
pranav[pranav["sessions_per_day"] == 24.531639758206442]

,active_timespan,ae_total_app_sessions,sessions_per_day,churned
distinct_id,,,,
11bc4edd4573c931,0.583333,14.310123,24.53164,0
1a28969580acaa3f,0.583333,14.310123,24.53164,0
4172008ac9320d3b,0.583333,14.310123,24.53164,0
50e9adab34f13549,0.583333,14.310123,24.53164,0
768bf0dcf15e5f1e,0.583333,14.310123,24.53164,0
7bf3d03785992b80,0.583333,14.310123,24.53164,0
8fe2d4985cbd2c17,0.583333,14.310123,24.53164,0
917a497f78326a7d,0.583333,14.310123,24.53164,0
A3BF314A-A411-4602-9774-0A4F4F3FD0AE,0.583333,14.310123,24.53164,0


In [70]:
original = data.copy()
original[original["distinct_id"] == "1a28969580acaa3f"]

,Unnamed: 0,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,FirstTimeUploaded,Signup With,imageAdded,commentAdded,commentLength,ImageUploaded,ae_total_app_session_length,name_y,ae_total_app_sessions,timezone,last_seen,ios_app_version,ios_version,ios_device_model,ios_lib_version,ae_first_app_open_date,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown
59695,25019,$ae_updated,1a28969580acaa3f,2019-08-30 05:17:28.000000,NaN,Crystal Lake,Illinois,US,1,1,NaN,Foodie,NaN,2,0.0,NaN,NaN,Natalie K,NaN,America/Detroit,2020-02-19 04:40:52+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,1,0,1,0,0
70325,39167,$ae_updated,1a28969580acaa3f,2019-08-06 14:18:28.000000,NaN,Crystal Lake,Illinois,US,1,0,NaN,Foodie,NaN,2,0.0,NaN,NaN,Natalie K,NaN,America/Detroit,2020-02-19 04:40:52+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,1,0,1,0,0
70326,39168,$ae_session,1a28969580acaa3f,2019-08-06 14:20:11.000000,102.6,Crystal Lake,Illinois,US,1,1,NaN,Foodie,NaN,2,0.0,NaN,NaN,Natalie K,NaN,America/Detroit,2020-02-19 04:40:52+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,1,0,1,0,0
93485,11009,Profile - Go to Menu,1a28969580acaa3f,2020-02-18 14:41:42.000000,NaN,Detroit,Michigan,US,1,1,NaN,Foodie,NaN,2,0.0,NaN,NaN,Natalie K,NaN,America/Detroit,2020-02-19 04:40:52+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,1,0,1,0,0
93498,10985,Visit Menu Page,1a28969580acaa3f,2020-02-18 14:41:44.000000,NaN,Detroit,Michigan,US,1,0,NaN,Foodie,NaN,2,0.0,NaN,NaN,Natalie K,NaN,America/Detroit,2020-02-19 04:40:52+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,1,0,1,0,0
93499,10986,AppOpen,1a28969580acaa3f,2020-02-18 14:40:52.000000,NaN,Detroit,Michigan,US,1,0,NaN,Foodie,NaN,2,0.0,NaN,NaN,Natalie K,NaN,America/Detroit,2020-02-19 04:40:52+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,1,0,1,0,0


In [71]:
original[original["distinct_id"] == "1a28969580acaa3f"]

,Unnamed: 0,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,FirstTimeUploaded,Signup With,imageAdded,commentAdded,commentLength,ImageUploaded,ae_total_app_session_length,name_y,ae_total_app_sessions,timezone,last_seen,ios_app_version,ios_version,ios_device_model,ios_lib_version,ae_first_app_open_date,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown
59695,25019,$ae_updated,1a28969580acaa3f,2019-08-30 05:17:28.000000,NaN,Crystal Lake,Illinois,US,1,1,NaN,Foodie,NaN,2,0.0,NaN,NaN,Natalie K,NaN,America/Detroit,2020-02-19 04:40:52+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,1,0,1,0,0
70325,39167,$ae_updated,1a28969580acaa3f,2019-08-06 14:18:28.000000,NaN,Crystal Lake,Illinois,US,1,0,NaN,Foodie,NaN,2,0.0,NaN,NaN,Natalie K,NaN,America/Detroit,2020-02-19 04:40:52+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,1,0,1,0,0
70326,39168,$ae_session,1a28969580acaa3f,2019-08-06 14:20:11.000000,102.6,Crystal Lake,Illinois,US,1,1,NaN,Foodie,NaN,2,0.0,NaN,NaN,Natalie K,NaN,America/Detroit,2020-02-19 04:40:52+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,1,0,1,0,0
93485,11009,Profile - Go to Menu,1a28969580acaa3f,2020-02-18 14:41:42.000000,NaN,Detroit,Michigan,US,1,1,NaN,Foodie,NaN,2,0.0,NaN,NaN,Natalie K,NaN,America/Detroit,2020-02-19 04:40:52+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,1,0,1,0,0
93498,10985,Visit Menu Page,1a28969580acaa3f,2020-02-18 14:41:44.000000,NaN,Detroit,Michigan,US,1,0,NaN,Foodie,NaN,2,0.0,NaN,NaN,Natalie K,NaN,America/Detroit,2020-02-19 04:40:52+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,1,0,1,0,0
93499,10986,AppOpen,1a28969580acaa3f,2020-02-18 14:40:52.000000,NaN,Detroit,Michigan,US,1,0,NaN,Foodie,NaN,2,0.0,NaN,NaN,Natalie K,NaN,America/Detroit,2020-02-19 04:40:52+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,1,0,1,0,0


In [72]:
pranav_og = dataset.copy()
pranav_og[pranav_og["distinct_id"] == "1a28969580acaa3f"]

,Unnamed: 0,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,FirstTimeUploaded,Signup With,imageAdded,commentAdded,commentLength,ImageUploaded,ae_total_app_session_length,name_y,ae_total_app_sessions,timezone,last_seen,ios_app_version,ios_version,ios_device_model,ios_lib_version,ae_first_app_open_date,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown
59695,25019,$ae_updated,1a28969580acaa3f,2020-02-18 14:40:52+00:00,NaN,Crystal Lake,Illinois,US,1,1,NaN,Foodie,NaN,2,0.0,NaN,NaN,Natalie K,14.310123,America/Detroit,2020-02-19 04:40:52+00:00,NaN,NaN,NaN,NaN,2020-02-18 14:40:52+00:00,14:00:00,NaN,US,1,0,1,0,0
70325,39167,$ae_updated,1a28969580acaa3f,2020-02-18 14:40:52+00:00,NaN,Crystal Lake,Illinois,US,1,0,NaN,Foodie,NaN,2,0.0,NaN,NaN,Natalie K,14.310123,America/Detroit,2020-02-19 04:40:52+00:00,NaN,NaN,NaN,NaN,2020-02-18 14:40:52+00:00,14:00:00,NaN,US,1,0,1,0,0
70326,39168,$ae_session,1a28969580acaa3f,2020-02-18 14:40:52+00:00,102.6,Crystal Lake,Illinois,US,1,1,NaN,Foodie,NaN,2,0.0,NaN,NaN,Natalie K,14.310123,America/Detroit,2020-02-19 04:40:52+00:00,NaN,NaN,NaN,NaN,2020-02-18 14:40:52+00:00,14:00:00,NaN,US,1,0,1,0,0
93485,11009,Profile - Go to Menu,1a28969580acaa3f,2020-02-18 14:40:52+00:00,NaN,Detroit,Michigan,US,1,1,NaN,Foodie,NaN,2,0.0,NaN,NaN,Natalie K,14.310123,America/Detroit,2020-02-19 04:40:52+00:00,NaN,NaN,NaN,NaN,2020-02-18 14:40:52+00:00,14:00:00,NaN,US,1,0,1,0,0
93498,10985,Visit Menu Page,1a28969580acaa3f,2020-02-18 14:40:52+00:00,NaN,Detroit,Michigan,US,1,0,NaN,Foodie,NaN,2,0.0,NaN,NaN,Natalie K,14.310123,America/Detroit,2020-02-19 04:40:52+00:00,NaN,NaN,NaN,NaN,2020-02-18 14:40:52+00:00,14:00:00,NaN,US,1,0,1,0,0
93499,10986,AppOpen,1a28969580acaa3f,2020-02-18 14:40:52+00:00,NaN,Detroit,Michigan,US,1,0,NaN,Foodie,NaN,2,0.0,NaN,NaN,Natalie K,14.310123,America/Detroit,2020-02-19 04:40:52+00:00,NaN,NaN,NaN,NaN,2020-02-18 14:40:52+00:00,14:00:00,NaN,US,1,0,1,0,0


In [73]:
erica[erica["distinct_id"] == "1a28969580acaa3f"]

,Unnamed: 0,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,FirstTimeUploaded,Signup With,imageAdded,commentAdded,commentLength,ImageUploaded,ae_total_app_session_length,name_y,ae_total_app_sessions,timezone,last_seen,ios_app_version,ios_version,ios_device_model,ios_lib_version,ae_first_app_open_date,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown,active_timespan_days,sessions_per_day
70325,39167,$ae_updated,1a28969580acaa3f,2019-08-06 14:18:28.000000,NaN,Crystal Lake,Illinois,US,1,0,NaN,Foodie,NaN,2,0.0,NaN,NaN,Natalie K,14.310123,America/Detroit,2020-02-19 04:40:52+00:00,NaN,NaN,NaN,NaN,2019-08-06 14:18:28+00:00,196 days 14:22:24,NaN,US,1,0,1,0,0,196.598889,0.072788


In [75]:
erica[erica["sessions_per_day"] < 0]

,Unnamed: 0,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,FirstTimeUploaded,Signup With,imageAdded,commentAdded,commentLength,ImageUploaded,ae_total_app_session_length,name_y,ae_total_app_sessions,timezone,last_seen,ios_app_version,ios_version,ios_device_model,ios_lib_version,ae_first_app_open_date,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown,active_timespan_days,sessions_per_day
77423,6827,$ae_updated,0038898ed9b23f7e,2019-08-23 05:22:36.000000,NaN,Taguig,Metro Manila,PH,1,1,NaN,Foodie,NaN,2,0.0,NaN,NaN,Unknown,14.310123,Asia/Manila,2019-05-24 04:51:53+00:00,NaN,NaN,NaN,NaN,2019-08-23 05:22:36+00:00,-92 days +23:29:17,NaN,INTL,0,0,0,0,1,-91.021331,-0.157217
92275,12147,$ae_updated,00471d582ed13800,2020-01-17 08:44:21.000000,NaN,Speedwell,Tennessee,US,1,0,NaN,Foodie,NaN,2,0.0,NaN,NaN,S F,14.310123,America/Los_Angeles,2019-06-06 19:56:50+00:00,NaN,NaN,NaN,NaN,2020-01-17 08:44:21+00:00,-225 days +11:12:29,NaN,US,1,0,1,0,0,-224.532998,-0.063733
94904,14154,$ae_updated,027e9256473c841d,2019-09-04 09:06:05.000000,NaN,Arlington,Virginia,US,1,0,NaN,Foodie,NaN,2,0.0,NaN,NaN,mona george,14.310123,America/New_York,2019-06-10 16:01:15+00:00,NaN,NaN,NaN,NaN,2019-09-04 09:06:05+00:00,-86 days +06:55:10,NaN,US,1,0,0,1,0,-85.711690,-0.166956
99440,45484,$ae_updated,0785BD4E-3F03-4069-B360-F5300B407605_305,2019-12-13 10:17:06.000000,NaN,Chicago,Illinois,US,1,0,NaN,Foodie,NaN,2,0.0,NaN,0.062,Unknown,2.000000,America/Chicago,2018-12-01 19:16:42+00:00,0.5.4,12.1,"iPhone9,3",3.3.3,2019-12-13 10:17:06+00:00,-377 days +08:59:36,31.0,US,1,1,0,0,0,-376.625278,-0.005310
95724,13296,$campaign_delivery,09719898d6bc68aa,2019-10-31 00:19:43.000000,NaN,NaN,NaN,NaN,1,1,NaN,Foodie,NaN,2,0.0,NaN,NaN,Bekki Filipczak,14.310123,America/New_York,2019-06-11 00:00:21+00:00,NaN,NaN,NaN,NaN,2019-10-31 00:19:43+00:00,-143 days +23:40:38,NaN,NaN,2,1,0,0,0,-142.013449,-0.100766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77773,6338,$ae_updated,siddharthseth96@gmail.com_240,2019-08-23 13:05:13.000000,NaN,NaN,Illinois,US,1,1,NaN,Foodie,NaN,2,0.0,NaN,NaN,Siddharth Seth,14.310123,America/Chicago,2019-07-08 22:37:16+00:00,NaN,NaN,NaN,NaN,2019-08-23 13:05:13+00:00,-46 days +09:32:03,NaN,US,1,0,1,0,0,-45.602743,-0.313800
96922,42531,$ae_session,suneelpasupuleti@ymail.com_4702,2019-08-18 13:24:05.000000,18.8,Sacramento,California,US,0,1,NaN,Foodie,NaN,2,0.0,NaN,NaN,Suneel Pasupuleti,14.310123,America/Los_Angeles,2019-07-01 19:28:02+00:00,NaN,NaN,NaN,NaN,2019-08-18 13:24:05+00:00,-48 days +06:03:57,NaN,US,1,1,0,0,0,-47.747257,-0.299706
67217,23336,$ae_updated,tiffanykim@uchicago_254,2019-08-25 10:02:50.000000,NaN,Great Falls,Virginia,US,1,1,NaN,Foodie,NaN,2,0.0,NaN,NaN,Tiffany Kim,14.310123,America/Chicago,2019-06-11 00:15:29+00:00,NaN,NaN,NaN,NaN,2019-08-25 10:02:50+00:00,-76 days +14:12:39,NaN,US,1,0,1,0,0,-75.407882,-0.189770
70158,39112,$ae_updated,wandaandreu@gmail.com_5053,2019-08-06 13:00:22.000000,NaN,Flushing,New York,US,1,1,NaN,Foodie,NaN,2,0.0,NaN,NaN,Wanda Andreu,14.310123,America/New_York,2019-07-17 21:30:52+00:00,NaN,NaN,NaN,NaN,2019-08-06 13:00:22+00:00,-20 days +08:30:30,NaN,US,1,0,0,1,0,-19.645486,-0.728418
